Citations: Andrew Lucas: Ising formulations of many NP Problems. arXiv:1302.5843. 
https://arxiv.org/abs/1302.5843

Ising machine: Cliques

A clique is a subgroup of nodes in an undirected graph where each node is
connected via an edge to every other node. The NP-complete problem is whether a
clique of size K exists. 

In [64]:
import numpy as np
import scienceplots
import matplotlib.pyplot as plt
import numba
import math

In [65]:
N = 100
# 1 means in clique, -1 means not in click. 
def generate_binary_spins(N):
    spins = np.random.choice(np.array([-1, 1], dtype=np.int64), size=N) # 1D array, size. generates random sample
    print(spins)
    # xalpha = (sa + 1) / 2
    # generate binary grid. 1 if spin is 1, 0 if spin is -1
    xalpha = np.zeros(N, dtype=np.int64)
    for i in range(N):
        if spins[i] == 1:
            xalpha[i] = 1
        else:
            xalpha[i] = 0
    return xalpha

xalpha = generate_binary_spins(N)
print(xalpha)


[ 1  1  1 -1  1  1  1  1  1 -1  1 -1  1  1 -1 -1  1 -1  1  1  1  1  1  1
 -1  1  1  1 -1  1 -1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1  1 -1
 -1 -1  1 -1  1  1 -1  1  1  1  1  1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1
 -1  1  1  1 -1  1  1 -1  1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1
  1 -1 -1  1]
[1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0
 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1
 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]


In [66]:
# Adjacency matrix for a graph with N nodes.
# N x N graph of 0 and 1 where 0 represents no connection, 1 represents a
# connection. 
# diagonal is all zereos. 

p = 0.9
@numba.njit
def build_adjacency_matrix(N, p):
    adjacency_matrix = np.zeros((N, N), dtype=np.int64)
    for i in range(N):
        for j in range(i + 1, N):
            if np.random.rand() < p and i != j:
                # Only add an edge if i and j are not the same node
                # and the random condition is met
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    return adjacency_matrix

adjacency_matrix = build_adjacency_matrix(N, p)
print("Adjacency Matrix:")
print(adjacency_matrix)

# if xu and xv are connected the it will be 1 in the adjacency matrix

def get_xuxv_matrix(edges, xalpha):
    # go through each edge in the adjacency matrix
    N = len(edges)
    xuxv_matrix = np.zeros((N, N), dtype=np.int64)
    for i in range(N):
        for j in range(i, N):
            if edges[i, j] == 1:
                # if there is an edge between i and j, then
                # xuxv = xalpha[i] * xalpha[j]
                xuxv_matrix[i, j] = xalpha[i] * xalpha[j]
    return xuxv_matrix

xuxv_matrix = get_xuxv_matrix(adjacency_matrix, xalpha)
print("xalpha")
print(xalpha)
print("xuxv Matrix:")
print(xuxv_matrix)

    

Adjacency Matrix:
[[0 1 1 ... 1 0 1]
 [1 0 1 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 ...
 [1 1 1 ... 0 1 0]
 [0 1 1 ... 1 0 1]
 [1 1 1 ... 0 1 0]]
xalpha
[1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0
 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1
 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]
xuxv Matrix:
[[0 1 1 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [67]:
# H = H(a) + H(b)
def calculate_hamiltonian(k, a, b, xuxv_matrix, xalpha):
    # h(a)
    h_a = a * (( k - np.sum(xalpha) )**2)
    # h(b)
    num_edges = (k * (k - 1)) / 2
    h_b = b * ( num_edges - np.sum(xuxv_matrix) )
    return h_a + h_b

# hamiltonian = calculate_hamiltonian(k, a, b, xuxv_matrix, xalpha)

In [102]:
def metropolis(xalpha, adjacency_matrix, k, a, b, T, steps):
    N = len(xalpha)
    xalpha = np.copy(xalpha)
    # Initialize the xuxv matrix
    xuxv = get_xuxv_matrix(adjacency_matrix, xalpha)
    # Calculate the initial Hamiltonian
    E_i = calculate_hamiltonian(k, a, b, xuxv, xalpha)
    for step in range(steps):
        # Choose a random node to flip to either 0 or 1
        i = np.random.randint(N)  # Pick a random node
        xalpha[i] = 1 - xalpha[i]
        # get the new xuxv_matrix 
        xuxv = get_xuxv_matrix(adjacency_matrix, xalpha)
        # Calculate the new Hamiltonian
        E_f = calculate_hamiltonian(k, a, b, xuxv, xalpha)
        # Calculate the energy difference
        delta_E = E_f - E_i
        # Metropolis acceptance criterion
        if delta_E < 0 or np.random.rand() < np.exp(-delta_E / T):
            # Accept the new state
            E_i = E_f
        else:
            # Reject the new state and revert the flip
            xalpha[i] = 1 - xalpha[i]
    return xalpha, E_i

T = 1.0
steps = 1000

def get_a(k, b):
    return (k * b) + 1

# Run the Metropolis algorithm for k 1 through N to see if a clique of size k
# exists. if it does, then the energy will be 0 (a solution to the equation)
def run(xalpha, adjacency_matrix, N, T, steps):
    b = 1
    k_results = []
    x_alphas = []
    for k in range(N+1):
        a = get_a(k,b)
        xalpha, E = metropolis(xalpha, adjacency_matrix, k, a, b, T, steps)
        k_results.append((k, E))
        x_alphas.append(xalpha)
    return k_results, x_alphas



n = 5
# random array of 0 and 1 of size n
xalpha_1 = np.random.randint(2, size=n)
print("xalpha_1:")
print(xalpha_1)
adjacency_matrix_1 = np.array([[0, 1, 1, 0, 0], 
                               [1, 0, 1, 1, 0], 
                               [1, 1, 0, 0, 0], 
                               [0, 1, 0, 0, 0], 
                               [0, 0, 0, 0, 0]])


k_results, x_alphas  = run(xalpha_1, adjacency_matrix_1, n, T, steps)
print("k results:")
for k, E in k_results:
    print(f"k: {k}, E: {E}")


print("x_alphas:")
for i, xalpha in enumerate(x_alphas):
    print(f"Iteration {i}: {xalpha}")
    







xalpha_1:
[0 1 0 0 0]
k results:
k: 0, E: 1.0
k: 1, E: 0.0
k: 2, E: 0.0
k: 3, E: 0.0
k: 4, E: 2.0
k: 5, E: 6.0
x_alphas:
Iteration 0: [1 0 0 0 0]
Iteration 1: [0 0 0 1 0]
Iteration 2: [1 1 0 0 0]
Iteration 3: [1 1 1 0 0]
Iteration 4: [1 1 1 1 0]
Iteration 5: [1 1 1 1 1]


In [ ]:
#print("Final xalpha:")
#print(xalpha_final)
#print("Final Energy:")
#print(final_energy)

Observation:

After many iterations, it is very hard to get and H = 0 for k = 0  (a clique of
size 0). I am not sure why this is. surely it would be possible to always get
the system to reduce to 0 items in the clique (e.g for n = 5, [0,0,0,0,0] ?).
On the other hand, it is (almost) always easy to output at least one solution and very fast
for each k = 1 through 3 for the undirected graph that I am using. This is a
very neat way of solving problems. It does rely on boltzmann sampling though
which isn't deterministic. Maybe if you repeat the experiment many many times
you can get something that is useful. I wouldn't trust my initial solution
though. AI